# Задание
Возьмите данные соревнования по определению перефразирования - http://paraphraser.ru/download/get?file_id=1 
Если не работает, то отсюда - https://drive.google.com/file/d/1rxm3K-0e1v4NYDzKq1Zi3FsXvh1yDakz/view?usp=sharing
Преобразуйте тексты в векторы в каждой паре 4 методами  - SVD, NMF, Word2Vec, Fastext. Для SVD и NMF сделайте две пары векторов - через TfidfVectorizer и CountVectorizer. Для word2vec сделайте две пары векторов - с взвешиванием по tfidf и без. Для Fastext постройте две модели - без нормализации и с нормализацией, а через каждую модель постройте две пары векторов -  с взвешиванием по tfidf и без. 

Для обучения этих моделей можете воспользоваться корпусом новостных текстов, с которым мы работали на семинаре. А можете использовать любой другой корпус (сами тексты соревнования использовать не надо).

У вас должно получиться 10 пар векторов для каждой строчки в датасете. Между векторами каждой пары вычислите косинусную близость (получится 10 чисел для каждой пары текстов). 

Постройте обучающую выборку из этих близостей. Обучите любую модель (Логрег, Рандом форест или что-то ещё) на этой выборке и оцените качество на кросс-валидации (используйте микросреднюю f1-меру).   

С помощью кросс-валидации подберите параметры моделей (количество компонент, размерность в w2v, min_n - в fastext и т.д). Перебирать все параметры и их сочетания не нужно! Можно перебрать несколько значений для какого-то параметра (например, 50, 100, 200 для svd), понять примерное влияние на метрику и переходить к следующему. Цель в этом пункте - найти 

Оценивание - если вы сделали всё вышеперечисленное - 10 баллов. Каждая ошибка - минус 0.5 балла. 


Выложите код к себе на гитхаб и вставьте ссылку в поле ниже (в тетрадке сохраните показатели метрик).

# Данные

In [1]:
from lxml import html
import pandas as pd
from string import punctuation
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))

In [2]:
def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0].normal_form for word in words if word and word not in stops]

    return ' '.join(words)

def tokenize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]

    return ' '.join(words)

### Данные, которые мы преобразуем

In [3]:
corpus_xml = html.fromstring(open('paraphrases.xml', 'rb').read())
texts_1 = []
texts_2 = []
classes = []

for p in corpus_xml.xpath('//paraphrase'):
    texts_1.append(p.xpath('./value[@name="text_1"]/text()')[0])
    texts_2.append(p.xpath('./value[@name="text_2"]/text()')[0])
    classes.append(p.xpath('./value[@name="class"]/text()')[0])
    
data = pd.DataFrame({'text_1':texts_1, 'text_2':texts_2, 'label':classes})

In [4]:
data['text_1_norm'] = data['text_1'].apply(normalize)
data['text_2_norm'] = data['text_2'].apply(normalize)

In [5]:
data.head()

,label,text_1,text_2,text_1_norm,text_2_norm
0,0,Полицейским разрешат стрелять на поражение по ...,Полиции могут разрешить стрелять по хулиганам ...,полицейский разрешить стрелять поражение гражд...,полиция мочь разрешить стрелять хулиган травма...
1,0,Право полицейских на проникновение в жилище ре...,Правила внесудебного проникновения полицейских...,право полицейский проникновение жилища решить ...,правило внесудебный проникновение полицейский ...
2,0,Президент Египта ввел чрезвычайное положение в...,Власти Египта угрожают ввести в стране чрезвыч...,президент египет ввести чрезвычайный положение...,власть египет угрожать ввести страна чрезвычай...
3,-1,Вернувшихся из Сирии россиян волнует вопрос тр...,Самолеты МЧС вывезут россиян из разрушенной Си...,вернуться сирия россиянин волновать вопрос тру...,самолёт мчс вывезти россиянин разрушить сирия
4,0,В Москву из Сирии вернулись 2 самолета МЧС с р...,Самолеты МЧС вывезут россиян из разрушенной Си...,москва сирия вернуться 2 самолёт мчс россиянин...,самолёт мчс вывезти россиянин разрушить сирия


### Данные для обучения модели

In [6]:
data_for_model = pd.read_csv('news_texts.csv')

In [7]:
data_for_model.head()

,content,content_norm
0,Канцлер Германии Ангела Меркель в ходе брифинг...,канцлер германия ангел меркель ход брифинг пре...
1,Российские и белорусские войска успешно заверш...,российский белорусский войско успешно завершит...
2,"Дзюба, Шатов и Анюков оказались не нужны «Зени...",дзюба шат анюк оказаться нужный зенит российск...
3,"В Испанию без фанатов\nПожалуй, главной пятнич...",испания фанат пожалуй главный пятничный новост...
4,"Постпред России при ООН Виталий Чуркин, говоря...",постпред россия оон виталий чуркин говорить ве...


In [8]:
data_for_model.isnull().values.any()

True

In [9]:
data_for_model = data_for_model.dropna()

### Преобразование в векторы

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD, NMF
import numpy as np
from gensim import matutils
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn import metrics

In [318]:
def show_result(X_text, y):
    train_X, valid_X, train_y, valid_y = train_test_split(X_text, y, random_state=1)
    clf = RandomForestClassifier(n_estimators=100, max_depth=10, min_samples_leaf=10,
                             class_weight='balanced')
    clf.fit(train_X, train_y)
    preds = clf.predict(valid_X)
    return 'F1-score: ' + str(metrics.f1_score(valid_y, preds, average='macro'))

In [319]:
y = data['label'].values

In [320]:
c_v = CountVectorizer(max_features=5000)
c_v = c_v.fit(data_for_model['content_norm'])
X_c_v = c_v.transform(data_for_model['content_norm'])

tfidf_v = TfidfVectorizer(max_features=5000)
tfidf_v = tfidf_v.fit(data_for_model['content_norm'])
X_tfidf_v = tfidf_v.transform(data_for_model['content_norm'])

In [321]:
models = {}

#### SVD

In [322]:
svd = TruncatedSVD(128)
svd.fit(X_c_v)
models['svd_c_v'] = svd

In [323]:
svd = TruncatedSVD(128)
svd.fit(X_tfidf_v)
models['svd_tfidf_v'] = svd

#### NMF

In [324]:
nmf = NMF(128)
nmf.fit(X_c_v)
models['nmf_c_v'] = nmf

In [325]:
nmf = NMF(128)
nmf.fit(X_tfidf_v)
models['nmf_tfidf_v'] = nmf

In [326]:
models

{'nmf_c_v': NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
   n_components=128, random_state=None, shuffle=False, solver='cd',
   tol=0.0001, verbose=0),
 'nmf_tfidf_v': NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
   n_components=128, random_state=None, shuffle=False, solver='cd',
   tol=0.0001, verbose=0),
 'svd_c_v': TruncatedSVD(algorithm='randomized', n_components=128, n_iter=5,
        random_state=None, tol=0.0),
 'svd_tfidf_v': TruncatedSVD(algorithm='randomized', n_components=128, n_iter=5,
        random_state=None, tol=0.0)}

In [327]:
def get_vectors(data, model, vctrzr):
    text_1 = data.text_1_norm
    text_2 = data.text_2_norm
    
    text2v_1 = vctrzr.transform(text_1)
    text2v_2 = vctrzr.transform(text_2)
    
    v2v_1 = model.transform(text2v_1)
    v2v_2 = model.transform(text2v_2)
    return v2v_1, v2v_2, [float(cosine_similarity([x], [y])) for x, y in zip(v2v_1, v2v_2)]

In [328]:
%%time
svd_c_v_1, svd_c_v_2, cos_svd_c_v = get_vectors(data, models['svd_c_v'], c_v)
nmf_c_v_1, nmf_c_v_2, cos_nmf_c_v = get_vectors(data, models['nmf_c_v'], c_v)
svd_tfidf_v_1, svd_tfidf_v_2, cos_svd_tfidf = get_vectors(data, models['svd_tfidf_v'], tfidf_v)
nmf_tfidf_v_1, nmf_tfidf_v_2, cos_nmf_tfidf = get_vectors(data, models['nmf_tfidf_v'], tfidf_v)

CPU times: user 11.8 s, sys: 277 ms, total: 12.1 s
Wall time: 12.1 s


##### NMF + CountVectorizer

In [329]:
X_text = np.concatenate([nmf_c_v_1, nmf_c_v_2], axis=1)

In [330]:
show_result(X_text, y)

'F1-score: 0.43192616842729853'

##### NMF + TfidfVectorizer

In [241]:
X_text = np.concatenate([nmf_tfidf_v_1, nmf_tfidf_v_2], axis=1)

In [242]:
show_result(X_text, y)

'F1-score: 0.43626449117140753'

##### SVD + CountVectorizer

In [243]:
X_text = np.concatenate([svd_c_v_1, svd_c_v_2], axis=1)

In [244]:
show_result(X_text, y)

'F1-score: 0.4455993288985442'

##### SVD + TfidfVectorizer

In [245]:
X_text = np.concatenate([svd_tfidf_v_1, svd_tfidf_v_2], axis=1)

In [246]:
show_result(X_text, y)

'F1-score: 0.4706772922077586'

#### Word2Vec

In [247]:
import gensim
from collections import Counter
from tqdm import tqdm_notebook as tqdm
import math

In [80]:
def compute_tf(text):
    tf_text = Counter(text)
    for i in tf_text:
        tf_text[i] = tf_text[i]/float(len(text))
    return tf_text

In [81]:
def compute_idf(word, corpus):
    return math.log10(len(corpus)/sum([1.0 for i in corpus if word in i]))

In [86]:
def compute_tfidf(corpus):
    documents_list = []
    for text in tqdm(corpus):
        tf_idf_dictionary = {}
        computed_tf = compute_tf(text)
        for word in computed_tf:
            tf_idf_dictionary[word] = computed_tf[word] * compute_idf(word, corpus)
        documents_list.append(tf_idf_dictionary)
    return documents_list

In [98]:
together = list(data['text_1_norm'].values) + list(data['text_2_norm'].values)
corpus = [text.split() for text in together]

In [99]:
tf_idf = compute_tfidf(corpus)

A Jupyter Widget

In [100]:
tf_idf_text_1, tf_idf_text_2 = tf_idf[:data.shape[0]], tf_idf[data.shape[0]:]

In [124]:
def get_embedding(text, model, dim, tfidf=False):
    text = text.split()
    words = Counter(text)
    total = len(text)
    vectors = np.zeros((len(words), dim))
    for i, word in enumerate(words):
        try:
            v = model[word]
            if tfidf:
                vectors[i] = v*tfidf[word]
            else:
                vectors[i] = v*(words[word]/total)
        except (KeyError, ValueError):
            continue
    if vectors.any():
        vector = np.average(vectors, axis=0)
    else:
        vector = np.zeros((dim))
    
    return vector

In [125]:
w2v = gensim.models.Word2Vec([text.split() for text in data_for_model['content_norm']], size=128, sg=1)

##### БЕЗ TFIDF

In [231]:
dim = 128
X_text_1_w2v = np.zeros((len(data['text_1_norm'].values), dim))
X_text_2_w2v = np.zeros((len(data['text_2_norm'].values), dim))

for i, text in enumerate(data['text_1_norm'].values):
    X_text_1_w2v[i] = get_embedding(text, w2v, dim)
    
for i, text in enumerate(data['text_2_norm'].values):
    X_text_2_w2v[i] = get_embedding(text, w2v, dim)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [232]:
X_text_w2v = np.concatenate([X_text_1_w2v, X_text_2_w2v], axis=1)

In [233]:
show_result(X_text_w2v, y)

'F1-score: 0.48849320026426746'

In [248]:
cos_w2v = [float(cosine_similarity([x], [y])) for x, y in zip(X_text_1_w2v, X_text_2_w2v)]

##### C TFIDF

In [249]:
dim = 128
X_text_1_w2v = np.zeros((len(data['text_1_norm'].values), dim))
X_text_2_w2v = np.zeros((len(data['text_2_norm'].values), dim))

for i, text in enumerate(data['text_1_norm'].values):
    X_text_1_w2v[i] = get_embedding(text, w2v, dim, tfidf=tf_idf_text_1[i])
    
for i, text in enumerate(data['text_2_norm'].values):
    X_text_2_w2v[i] = get_embedding(text, w2v, dim, tfidf=tf_idf_text_2[i])

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [250]:
X_text_w2v = np.concatenate([X_text_1_w2v, X_text_2_w2v], axis=1)

In [251]:
show_result(X_text_w2v, y)

'F1-score: 0.48043122317367576'

In [252]:
cos_w2v_tfidf = [float(cosine_similarity([x], [y])) for x, y in zip(X_text_1_w2v, X_text_2_w2v)]

#### FastText

In [49]:
fast_text = gensim.models.FastText([text.split() for text in data_for_model['content_norm']], size=128, min_n=4, max_n=8)

##### БЕЗ TFIDF, БЕЗ НОРМАЛИЗАЦИИ

In [291]:
dim = 128
data['text_1_notnorm'] = data['text_1'].apply(tokenize)
data['text_2_notnorm'] = data['text_2'].apply(tokenize)

X_text_1_ft = np.zeros((len(data['text_1_notnorm']), dim))
X_text_2_ft = np.zeros((len(data['text_2_notnorm']), dim))

for i, text in enumerate(data['text_1_notnorm'].values):
    X_text_1_ft[i] = get_embedding(text, fast_text, dim)
    
for i, text in enumerate(data['text_2_notnorm'].values):
    X_text_2_ft[i] = get_embedding(text, fast_text, dim)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [292]:
X_text_ft = np.concatenate([X_text_1_ft, X_text_2_ft], axis=1)

In [293]:
show_result(X_text_ft, y)

'F1-score: 0.4721337121674352'

In [294]:
cos_ft = [float(cosine_similarity([x], [y])) for x, y in zip(X_text_1_ft, X_text_2_ft)]

##### БЕЗ TFIDF, C НОРМАЛИЗАЦИЕЙ

In [295]:
dim = 128
X_text_1_ft = np.zeros((len(data['text_1_norm'].values), dim))
X_text_2_ft = np.zeros((len(data['text_2_norm'].values), dim))

for i, text in enumerate(data['text_1_norm'].values):
    X_text_1_ft[i] = get_embedding(text, fast_text, dim)
    
for i, text in enumerate(data['text_2_norm'].values):
    X_text_2_ft[i] = get_embedding(text, fast_text, dim)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [296]:
X_text_ft = np.concatenate([X_text_1_ft, X_text_2_ft], axis=1)

In [297]:
show_result(X_text_ft, y)

'F1-score: 0.49046982893835245'

In [298]:
cos_ft_norm = [float(cosine_similarity([x], [y])) for x, y in zip(X_text_1_ft, X_text_2_ft)]

##### C TFIDF, C НОРМАЛИЗАЦИЕЙ

In [299]:
dim = 128
X_text_1_ft = np.zeros((len(data['text_1_norm'].values), dim))
X_text_2_ft = np.zeros((len(data['text_2_norm'].values), dim))

for i, text in enumerate(data['text_1_norm'].values):
    X_text_1_ft[i] = get_embedding(text, fast_text, dim, tfidf=tf_idf_text_1[i])
    
for i, text in enumerate(data['text_2_norm'].values):
    X_text_2_ft[i] = get_embedding(text, fast_text, dim, tfidf=tf_idf_text_2[i])

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [300]:
X_text_ft = np.concatenate([X_text_1_ft, X_text_2_ft], axis=1)

In [301]:
show_result(X_text_ft, y)

'F1-score: 0.47838389271955695'

In [302]:
cos_ft_norm_tfidf = [float(cosine_similarity([x], [y])) for x, y in zip(X_text_1_ft, X_text_2_ft)]

##### C TFIDF, БЕЗ НОРМАЛИЗАЦИИ

In [303]:
dim = 128
X_text_1_ft = np.zeros((len(data['text_1_notnorm']), dim))
X_text_2_ft = np.zeros((len(data['text_2_notnorm']), dim))

for i, text in enumerate(data['text_1_notnorm'].values):
    X_text_1_ft[i] = get_embedding(text, fast_text, dim, tfidf=tf_idf_text_1[i])
    
for i, text in enumerate(data['text_2_notnorm'].values):
    X_text_2_ft[i] = get_embedding(text, fast_text, dim, tfidf=tf_idf_text_2[i])

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [304]:
X_text_ft = np.concatenate([X_text_1_ft, X_text_2_ft], axis=1)

In [305]:
show_result(X_text_ft, y)

'F1-score: 0.4715568184675736'

In [306]:
cos_ft_tfidf = [float(cosine_similarity([x], [y])) for x, y in zip(X_text_1_ft, X_text_2_ft)]

In [331]:
# create dataset
d = {'cos_svd_c_v': cos_svd_c_v, 
     'cos_nmf_c_v': cos_nmf_c_v, 
     'cos_svd_tfidf': cos_svd_tfidf, 
     'cos_nmf_tfidf': cos_nmf_tfidf, 
     'cos_w2v': cos_w2v, 
     'cos_w2v_tfidf': cos_w2v_tfidf, 
     'cos_ft': cos_ft, 
     'cos_ft_tfidf': cos_ft_tfidf, 
     'cos_ft_norm_tfidf': cos_ft_norm_tfidf, 
     'cos_ft_norm': cos_ft_norm}

In [332]:
X = pd.DataFrame(data=d)

In [333]:
X.head()

,cos_ft,cos_ft_norm,cos_ft_norm_tfidf,cos_ft_tfidf,cos_nmf_c_v,cos_nmf_tfidf,cos_svd_c_v,cos_svd_tfidf,cos_w2v,cos_w2v_tfidf
0,0.776740,0.697593,0.715177,0.644239,0.521702,0.931057,0.137964,0.627718,0.849223,0.849223
1,0.743313,0.772822,0.779357,0.000000,0.194543,0.920925,0.395494,0.738407,0.872423,0.872423
2,0.668038,0.802900,0.812663,0.632182,0.007293,0.893944,0.094306,0.709270,0.928296,0.928296
3,0.683037,0.595254,0.581431,-0.057154,0.511020,0.897832,0.575513,0.689159,0.644343,0.644343
4,0.603453,0.698070,0.634781,0.395326,0.949005,0.957088,0.888962,0.962167,0.859328,0.859328


In [334]:
show_result(X, y)

'F1-score: 0.5674339364884144'

# Перебор параметров

```
SVD: [50, 100, 200]
NMF: [50, 100, 200]
w2v: size=[50, 100, 200]
FastText: size=[50, 100, 200], max_n=[8, 15]
```

In [335]:
cos_svd = []
for x in tqdm([50, 100, 200]):
    svd = TruncatedSVD(x)
    svd.fit(X_c_v)
    svd_c_v_1, svd_c_v_2, cos_svd_c_v = get_vectors(data, svd, c_v)
    cos_svd.append(cos_svd_c_v)
for x in tqdm([50, 100, 200]):
    svd = TruncatedSVD(x)
    svd.fit(X_tfidf_v)
    svd_tfidf_v_1, svd_tfidf_v_2, cos_svd_tfidf = get_vectors(data, svd, tfidf_v)
    cos_svd.append(cos_svd_tfidf)

A Jupyter Widget

A Jupyter Widget

In [336]:
for each in cos_svd[:3]:
    X['cos_svd_c_v'] = each
    print(show_result(X, y))

F1-score: 0.5637384168849026
F1-score: 0.5686350530143768
F1-score: 0.5678713021371428


In [337]:
# лучше размерность 100
X['cos_svd_c_v'] = cos_svd[1]

In [338]:
for each in cos_svd[3:]:
    X['cos_svd_tfidf'] = each
    print(show_result(X, y))

F1-score: 0.560032037788467
F1-score: 0.5749721513666372
F1-score: 0.5698658885325582


In [339]:
# лучше размерность 100
X['cos_svd_tfidf'] = cos_svd[4]

In [341]:
cos_nmf = []
for x in tqdm([50, 100, 200]):
    nmf = NMF(x)
    nmf.fit(X_c_v)
    nmf_c_v_1, nmf_c_v_2, cos_nmf_c_v = get_vectors(data, nmf, c_v)
    cos_nmf.append(cos_nmf_c_v)
for x in tqdm([50, 100, 200]):
    nmf = NMF(x)
    nmf.fit(X_tfidf_v)
    nmf_tfidf_v_1, nmf_tfidf_v_2, cos_nmf_tfidf = get_vectors(data, nmf, tfidf_v)
    cos_nmf.append(cos_nmf_tfidf)

A Jupyter Widget

A Jupyter Widget

In [342]:
for each in cos_svd[:3]:
    X['cos_nmf_c_v'] = each
    print(show_result(X, y))

F1-score: 0.5665590124120888
F1-score: 0.5686495605521474
F1-score: 0.5703847337146141


In [343]:
# лучше размерность 200
X['cos_nmf_c_v'] = cos_nmf[2]

In [344]:
for each in cos_svd[3:]:
    X['cos_nmf_tfidf'] = each
    print(show_result(X, y))

F1-score: 0.5778116437242705
F1-score: 0.5776897594274245
F1-score: 0.5775658317946109


In [345]:
# лучше размерность 50
X['cos_nmf_tfidf'] = cos_nmf[3]

In [346]:
cos_w2v_l = []
for x in tqdm([50, 100, 200]):
    w2v = gensim.models.Word2Vec([text.split() for text in data_for_model['content_norm']], size=x, sg=1)
    dim = x
    X_text_1_w2v = np.zeros((len(data['text_1_norm'].values), dim))
    X_text_2_w2v = np.zeros((len(data['text_2_norm'].values), dim))

    for i, text in enumerate(data['text_1_norm'].values):
        X_text_1_w2v[i] = get_embedding(text, w2v, dim)
    
    for i, text in enumerate(data['text_2_norm'].values):
        X_text_2_w2v[i] = get_embedding(text, w2v, dim)
    cos_w2v = [float(cosine_similarity([x], [y])) for x, y in zip(X_text_1_w2v, X_text_2_w2v)]
    cos_w2v_l.append(cos_w2v)
for x in tqdm([50, 100, 200]):
    w2v = gensim.models.Word2Vec([text.split() for text in data_for_model['content_norm']], size=x, sg=1)
    dim = x
    X_text_1_w2v = np.zeros((len(data['text_1_norm'].values), dim))
    X_text_2_w2v = np.zeros((len(data['text_2_norm'].values), dim))

    for i, text in enumerate(data['text_1_norm'].values):
        X_text_1_w2v[i] = get_embedding(text, w2v, dim, tfidf=tf_idf_text_1[i])
    
    for i, text in enumerate(data['text_2_norm'].values):
        X_text_2_w2v[i] = get_embedding(text, w2v, dim, tfidf=tf_idf_text_2[i])
    cos_w2v = [float(cosine_similarity([x], [y])) for x, y in zip(X_text_1_w2v, X_text_2_w2v)]
    cos_w2v_l.append(cos_w2v)

A Jupyter Widget

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


A Jupyter Widget

In [351]:
for each in cos_w2v_l[:3]:
    X['cos_w2v'] = each
    print(show_result(X, y))

F1-score: 0.5651272112522348
F1-score: 0.5661895218715021
F1-score: 0.5632773151419366


In [352]:
# лучше размерность 100
X['cos_w2v'] = cos_w2v_l[1]

In [353]:
for each in cos_w2v_l[3:]:
    X['cos_w2v_tfidf'] = each
    print(show_result(X, y))

F1-score: 0.5679348653121549
F1-score: 0.5674843354379409
F1-score: 0.5656971672415686


In [354]:
# лучше размерность 50
X['cos_w2v_tfidf'] = cos_svd[3]

In [355]:
# берём лучший результат с FastText и параметры будем подбирать только тут

In [ ]:
cos_ft_l = []
# с нормализацией, без tfidf
for x in tqdm([50, 100, 200]):
    for y in [8, 15]:
        print(x, y)
        fast_text = gensim.models.FastText([text.split() for text in data_for_model['content_norm']], size=x, min_n=4, max_n=y)
        dim = x
        X_text_1_ft = np.zeros((len(data['text_1_norm'].values), dim))
        X_text_2_ft = np.zeros((len(data['text_2_norm'].values), dim))

        for i, text in enumerate(data['text_1_norm'].values):
            X_text_1_ft[i] = get_embedding(text, fast_text, dim)
    
        for i, text in enumerate(data['text_2_norm'].values):
            X_text_2_ft[i] = get_embedding(text, fast_text, dim)
        cos_ft_tfidf = [float(cosine_similarity([x1], [y1])) for x1, y1 in zip(X_text_1_ft, X_text_2_ft)]
        cos_ft_l.append(cos_ft_tfidf)
        print(cos_ft_tfidf)

A Jupyter Widget

50 8


In [357]:
for each in cos_ft_l:
    X['cos_ft_norm'] = each
    print(show_result(X, y))

TypeError: Singleton array array(15) cannot be considered a valid collection.

In [ ]:
# лучше размерность и max_n=
X['cos_ft_norm'] = cos_ft_l[]

# Результат

In [ ]:
show_result(X, y)